In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Import libraries
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained("Your_checkpoint", device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

output_merged_dir = "results/llama2/final_merged_checkpoint"
os.makedirs(output_merged_dir, exist_ok=True)
model.save_pretrained(output_merged_dir, safe_serialization=True)

# save tokenizer for easy inference
tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")
tokenizer.save_pretrained(output_merged_dir)

In [ ]:
text = """
### Instruction:
Below are the details of the legal case. Based on this information, can you anticipate the verdict? If the judgment favors the plaintiff or the defendant, what might the penalty be? In the event of a settlement, what could be the agreed-upon compensation amount?

### Input:
Title: Blizzard et al v. Delta Air Lines, Inc
Docket_header: U.S. District Court Middle District of Florida (Tampa) CIVIL DOCKET FOR CASE #: 8:05-cv-00026-SCB-TGW
Date_filed: 01/05/2005
Jurisdiction: Diversity
Cause: 28:1332 Diversity-Personal Injury
Nature_of_suit: 310 Airplane
Judge: Judge Susan C Bucklew
Referred_to: Magistrate Judge Thomas G. Wilson
Plaintiff_and_attorneys: [{'plaintiff_name': 'Mary A. Blizzard', 'attorneys': ['Richard A. Hirsch', 'Stephen Lutz Segall']}, {'plaintiff_name': 'William D. Blizzard, Jr.', 'attorneys': ['Richard A. Hirsch', 'Stephen Lutz Segall']}]
Defendant_and_attorneys: [{'defendant_name': 'Delta Air Lines, Inc', 'attorneys': ['James Scott McMahon', 'Richard K. Bowers , Jr.']}]
"""

# response:
# Based on the information provided, the outcome of the case is a settlement with a stipulation of dismissal with prejudice. However, the compensation amount is not disclosed. Therefore, the result of the case is a settlement with a stipulation of dismissal with prejudice.


# Specify device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Tokenize input text
inputs = tokenizer(text, return_tensors="pt").to(device)

# Get answer
# (Adjust max_new_tokens variable as you wish (maximum number of tokens the model can generate to answer the input))
outputs = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"], max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

# Decode output & print it
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Instruction:
Below are the details of the legal case. Based on this information, can you anticipate the verdict? If the judgment favors the plaintiff or the defendant, what might the penalty be? In the event of a settlement, what could be the agreed-upon compensation amount?

### Input:
Title: Blizzard et al v. Delta Air Lines, Inc
Docket_header: U.S. District Court Middle District of Florida (Tampa) CIVIL DOCKET FOR CASE #: 8:05-cv-00026-SCB-TGW
Date_filed: 01/05/2005
Jurisdiction: Diversity
Cause: 28:1332 Diversity-Personal Injury
Nature_of_suit: 310 Airplane
Judge: Judge Susan C Bucklew
Referred_to: Magistrate Judge Thomas G. Wilson
Plaintiff_and_attorneys: [{'plaintiff_name': 'Mary A. Blizzard', 'attorneys': ['Richard A. Hirsch', 'Stephen Lutz Segall']}, {'plaintiff_name': 'William D. Blizzard, Jr.', 'attorneys': ['Richard A. Hirsch', 'Stephen Lutz Segall']}]
Defendant_and_attorneys: [{'defendant_name': 'Delta Air Lines, Inc', 'attorneys': ['James Scott McMahon', 'Richard K. B